In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [40]:
# df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = pd.read_csv("/data/funds.csv")

In [24]:
# df.to_csv("/data/funds.csv", index=False)

In [41]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID000001,Vallie Bachman,The University of Sydney,Female,Hobart,Tasmania,N,FID000106,NOT APPLICABLE,-42.8806,...,44,Middle-aged,21,MORE_THAN_10_YEAR,1,NaN,1980-01-08,2004-11-18,9999-12-31,2045-01-08
1,MID000028,Jani Herritt,Murdoch University,Female,Sydney,New South Wales,Y,FID000137,NOT APPLICABLE,-33.8678,...,52,Senior,13,MORE_THAN_10_YEAR,1,NaN,1972-02-27,2001-07-08,9999-12-31,2037-02-27
2,MID000044,Jani Rulapaugh,Torrens University Australia,Male,Melbourne,Victoria,Y,FID000050,NOT APPLICABLE,-37.8142,...,43,Middle-aged,22,MORE_THAN_10_YEAR,1,NaN,1981-02-17,2002-12-10,9999-12-31,2046-02-17
3,MID000089,Alease Hollack,University of New England,Male,Darwin,Northern Territory,N,FID000186,NOT APPLICABLE,-12.4381,...,43,Middle-aged,22,MORE_THAN_10_YEAR,1,NaN,1981-12-19,2002-08-13,9999-12-31,2046-12-19
4,MID000098,Devorah Whobrey,The University of Notre Dame Australia,Female,Darwin,Northern Territory,Y,FID000184,NOT APPLICABLE,-12.4381,...,59,Senior,6,IN_NEXT_5_YEAR,1,NaN,1965-12-07,2005-08-05,9999-12-31,2030-12-07


In [42]:
df.isna().sum() / df.shape[0]

MEMBER_ID                                           0.000000
MEMBER_NAME                                         0.000000
MEMBER_EMPLOYMENT                                   0.000000
MEMBER_GENDER                                       0.000000
MEMBER_CITY_TOWN                                    0.000000
MEMBER_STATE                                        0.000000
MEMBER_CONTACT_VERIFIED                             0.000000
FUND_ID                                             0.000000
CHURN_REASON                                        0.000000
LATITUDE                                            0.000000
LONGITUDE                                           0.000000
COUNTRY                                             0.000000
COUNTRYCODE                                         0.000000
CAPITAL                                             0.000000
POPULATION                                          0.000000
FUND_TOTAL_ASSETS                                   0.000000
FUND_RETURN_TARGET_PERCE

NO_OF_CHURN column has almost 50% missing value, it wont be wise to impute these many values so will just drop it.                                     

In [86]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [87]:
frame["CHURN_FLAG"] = frame["CHURN_FLAG"].apply(lambda x: 1 if x =="Y" else 0)
# data

In [82]:
# frame.groupby(["RETIREMENT_AGE_GROUP","CHURN_FLAG"])[["CHURN_FLAG"]].count()

The distribution of CHURN is pretty common bettween the employers and hence wont contribute inn modelling

In [76]:
# multiple = {i for i, j in dict(df["MEMBER_ID"].value_counts()).items() if j > 1 }
# multiple[multiple["value_counts"]>=2].index.to_list()

In [88]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

np.random.seed(0)

,MEMBER_GENDER,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_TOTAL_ASSETS,FUND_RETURN_TARGET_PERCENTAGE,INVESTMENT_RISK_CATEGORY,CASH_BENCHMARK_ALLOCATION,FIXED_INCOME_BENCHMARK_ALLOCATION,DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION,INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION,...,YEAR_3_RETURNS,YEAR_5_RETURNS,YEAR_7_RETURNS,YEAR_10_RETURNS,SUPER_FEES,PENSION_FEES,INVESTMENT_AGE_GROUP,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,CHURN_FLAG
0,Female,Tasmania,N,203653.1,4.229630,Medium-Low,4.651852,20.111111,26.500000,26.296296,...,4.83,4.62,5.24,5.71,0.53,0.46,Middle-aged,MORE_THAN_10_YEAR,1,0
1,Female,New South Wales,Y,11341.0,3.500000,Medium-High,5.000000,29.166667,31.333333,25.000000,...,4.42,4.39,5.03,5.48,0.51,0.42,Senior,MORE_THAN_10_YEAR,1,0
2,Male,Victoria,Y,919979.7,4.526786,Low,3.000000,17.940741,25.981481,21.633333,...,3.45,2.37,2.63,2.54,0.13,0.13,Middle-aged,MORE_THAN_10_YEAR,1,0
3,Male,Northern Territory,N,3102.3,4.122222,Low,10.000000,10.000000,10.000000,10.000000,...,3.10,2.19,2.25,2.95,0.14,0.14,Middle-aged,MORE_THAN_10_YEAR,1,0
4,Female,Northern Territory,Y,1313.2,2.600000,Medium-High,1.000000,29.000000,0.000000,0.000000,...,4.07,4.07,5.06,5.72,0.53,0.45,Senior,IN_NEXT_5_YEAR,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335994,Female,New South Wales,Y,35810.9,3.178571,High,5.000000,18.000000,25.000000,0.000000,...,4.55,5.25,7.11,7.77,0.55,0.56,Senior,IN_NEXT_10_YEAR,2,1
335995,Female,Queensland,Y,12332.5,3.000000,Very High,7.500000,20.500000,0.000000,0.000000,...,5.46,8.34,9.42,9.50,0.45,0.39,Young,MORE_THAN_10_YEAR,2,1
335996,Male,South Australia,N,203653.1,4.229630,Medium-Low,4.651852,20.111111,26.500000,26.296296,...,4.83,4.62,5.24,5.71,0.53,0.46,Middle-aged,MORE_THAN_10_YEAR,2,1
335997,Female,New South Wales,Y,12332.5,3.000000,Very High,7.500000,20.500000,0.000000,0.000000,...,5.46,8.34,9.42,9.50,0.45,0.39,Middle-aged,MORE_THAN_10_YEAR,2,1
